In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import numpy as np
import os

In [ ]:
# import fields and grid

%store -r fields
[uWind,vWind,wWind,temp,geop,div,vort,geop_height] = fields

%store -r month

script_dir = os.path.dirname(__name__)
rel_path = f'files/2020_{month}_01_hourly.nc'
abs_file_path = os.path.join(script_dir, rel_path)
rel_path_surf = f'files/2020_{month}_01_hourly_surf.nc'
abs_file_path_surf = os.path.join(script_dir, rel_path_surf)

data = xr.open_dataset(abs_file_path)
rel_hum = data['r']
lon = data.longitude
lat = data.latitude
pressure_levels = np.array(data.level)*100

In [ ]:
# for calculating the gradient of a given field, first the distance between
# grid points is converted to metres. the distance in meridional direction is
# constant (deg_to_m) but varies in zonal direction depending on the latitude (dist)

deg_to_m = 111120

dist = np.array(deg_to_m * np.cos(np.deg2rad(lat)))
dist = np.swapaxes(np.tile(dist,[len(lon),1]),0,1)

def grad(field):
    '''this function calculates the gradient of field'''
    grad = np.zeros((2,*temp.isel(time=0).shape))
    
    for j in range(1,len(field.latitude)-1):
        grad[1,:,j,:] = - 1 / (2 * deg_to_m) * (field.isel(time=1)[:,j+1,:] - field.isel(time=1)[:,j-1,:])
    grad[1,:,0,:] = - 1 / deg_to_m * (field.isel(time=1)[:,1,:] - field.isel(time=1)[:,0,:])
    grad[1,:,-1,:] = - 1 / deg_to_m * (field.isel(time=1)[:,-1,:] - field.isel(time=1)[:,-2,:])
    
    for i in range(1,len(field.longitude)-1):
        grad[0,:,:,i] = 1 / (2 * dist[:,i]) * (field.isel(time=1)[:,:,i+1] - field.isel(time=1)[:,:,i-1])
    grad[0,:,:,0] = 1 / dist[:,0] * (field.isel(time=1)[:,:,1] - field.isel(time=1)[:,:,0])
    grad[0,:,:,-1] = 1 / dist[:,-1] * (field.isel(time=1)[:,:,-1] - field.isel(time=1)[:,:,-2])
    
    return grad

In [ ]:
# import coriolis parameter

%store -r f

In [ ]:
# for calculating the virtual temperature (used later in the calculation
# of the air density) first the vapor pressure of water is calculated
# using the clausius-clapeyron equation

# some constants
Lv = 2265e3 # latent heat of vaporization
M = 28.9647e-3 # molar mass of air
R = 8.314 # ideal gas constant
T0 = 273.15 + 30 # a random reference temperature
e0 = 4.2455 # the vapor pressure at the reference temperature

# vapor pressure
vap_press = e0 * np.exp(Lv * M * (temp - T0)/(R * T0 * temp))
# partial pressure of water in air
part_press = rel_hum * vap_press
# virtual temperature
temp_v = xr.zeros_like(temp)
for i in range(len(pressure_levels)):
    temp_v[:,i,:,:] = temp[:,i,:,:] / (1 - part_press[:,i,:,:]/pressure_levels[i] * (1 - 0.622))

The vorticity equation (neglecting friction) is
\begin{align}
\Leftrightarrow \qquad \dfrac{\partial}{\partial t} \eta + \pmb{u}_h \nabla \eta + w_p \dfrac{\partial}{\partial p} \eta &= - \eta \nabla \pmb{u} - \pmb{k} \left( \nabla w_p \times \dfrac{\partial \pmb{u}}{\partial p} \right) + \rho \pmb{k} ( \nabla \alpha \times \nabla \Phi )
\end{align}
with $\nabla$ and $w_p$ given in pressure coordinates <br>
The terms are (from left to right): local change of vorticity, horizontal vorticity advection, vertical vorticity advection, stretching of vortivity due to flow compressibility (ballerina effect), tilting term, and solenoid term

In [ ]:
# calculate the individual terms of the vorticity equation

# specific gas constant of air
R_spec = 8.1343 / 0.0289647

# absolute vorticity (in the following just called vorticity)
eta = vort + f

# calculate air density from ideal gas equation using the virtual temperature
rho = xr.zeros_like(eta)
for p in range(len(pressure_levels)):
    rho[:,p,:,:] = pressure_levels[p] / (R_spec * temp_v[:,p,:,:])
alpha = 1 / rho

# local change of vorticity
loc_change_eta = ( eta[2,:,:,:] - eta[0,:,:,:] ) / 7200

# horizontal (i.e. on pressure levels) vorticity advection
hor_adv_eta = uWind[1,:,:,:] * grad(eta)[0] + vWind[1,:,:,:] * grad(eta)[1]

# vertical vorticity advection
vert_adv_eta = xr.zeros_like(eta)
for p in range(1,len(pressure_levels)):
    vert_adv_eta[:,p,:,:] = wWind[:,p,:,:] * ( eta[:,p,:,:] - eta[:,p-1,:,:] ) \
                            / ( pressure_levels[p] - pressure_levels[p-1] )
vert_adv_eta = vert_adv_eta[1,:,:,:]

# ballerina effect
ball_term = - eta[1,:,:,:] * ( grad(uWind)[0] + grad(vWind)[1] )

# tilting term
tilt_term = xr.zeros_like(eta)
for p in range(1,len(pressure_levels)):
    tilt_term[:,p,:,:] = - ( grad(wWind)[0,p,:,:] * (vWind[1,p,:,:] - vWind[1,p-1,:,:]) / (pressure_levels[p] - pressure_levels[p-1]) \
                           - grad(wWind)[1,p,:,:] * (uWind[1,p,:,:] - uWind[1,p-1,:,:]) / (pressure_levels[p] - pressure_levels[p-1]))
tilt_term = tilt_term[1,:,:,:]

# solenoid term
sol_term = - rho[1,:,:,:] * (grad(alpha)[0] * grad(geop)[1] - grad(alpha)[1] * grad(geop)[0])
sol_term = xr.DataArray(sol_term)

In [ ]:
# add name and units to the fields (for plotting)

eta.attrs['long_name'] = 'absolute vorticity'
eta.attrs['units'] = 's**-1'
loc_change_eta.attrs['long_name'] = 'local change of vorticity'
loc_change_eta.attrs['units'] = 's**-1'
hor_adv_eta.attrs['long_name'] = 'horizontal vorticity advection'
hor_adv_eta.attrs['units'] = 's**-1'
vert_adv_eta.attrs['long_name'] = 'vertical vorticity advection'
vert_adv_eta.attrs['units'] = 's**-1'
ball_term.attrs['long_name'] = 'ballerina effect'
ball_term.attrs['units'] = 's**-1'
tilt_term.attrs['long_name'] = 'tilting term'
tilt_term.attrs['units'] = 's**-1'
sol_term.attrs['long_name'] = 'solenoid term'
sol_term.attrs['units'] = 's**-1'

In [ ]:
def plot_vort_terms(field,N=90,S=90,W=0,E=360,pressure_level=0,
                    spacing=5,vmin=None,vmax=None,wind=False):
    '''to be used for the scale analysis of the vorticity equation.
       this function plots field with an areal extend of [N,S,W,E] at pressure_level.
       W is given in degrees east and has to be smaller than E,
       also no negative values are allowed.
       the geopotential height is displayed as contour lines.
       vmin and vmax are the minumum and maximum values of field in the colorbar.
       wind is a flag whether the wind field is displayed as arrows or not.
    '''

    N = 90-N
    S = 90+S
    
    fig, ax = plt.subplots(figsize=(15,8), subplot_kw={'projection': ccrs.PlateCarree()})
    im = ax.contourf(lon[W:E], lat[N:S], field[pressure_level,N:S,W:E],
                    cmap='viridis', levels=50, vmin=vmin, vmax=vmax)
    
    im2 = ax.contour(lon[W:E], lat[N:S], geop_height[1,pressure_level,N:S,W:E])
    ax.clabel(im2, im2.levels, inline=True,colors='k')
    
    if wind:
        Q = ax.quiver(lon[W:E][::spacing], lat[N:S][::spacing],
                    uWind[1,pressure_level,N:S,W:E][::spacing,::spacing],
                    vWind[1,pressure_level,N:S,W:E][::spacing,::spacing])
        Qk = ax.quiverkey(Q,0.5,-0.1,np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing]),
                label="{:.0f}".format(np.array(np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing]))) + " m/s wind velocity",
                labelpos = 'E')

    ax.add_feature(cf.COASTLINE)
    ax.add_feature(cf.BORDERS)
    ax.set_xticks([0],[0])
    ax.set_yticks([0],[0])

    fig.colorbar(im, orientation='horizontal', fraction=0.039*len(lon)/len(lat), label=f"{field.long_name} [{field.units}]")
    ax.set_title(f"{field.long_name} at pressure level {int(pressure_levels[pressure_level]/100)} hPa", fontsize=15)
    fig.tight_layout()

### Use the function to display and compare the terms of the vorticity equation

try out different regions and altitudes <br>
<br>
the variables are <br>
local change of vorticity: loc_change_eta <br>
horizontal (i.e. on pressure levels) vorticity advection: hor_adv_eta <br>
vertical vorticity advection: vert_adv_eta <br>
ballerina effect: ball_term <br>
tilting term: tilt_term <br>
solenoid term: sol_term

In [ ]:
pressure_levels/100 # in hPa

In [ ]:
plot_vort_terms(sol_term,N=70,S=10,E=360,W=290,pressure_level=-2,wind=False)